## Métodos Privados

#### 9.6. Variáveis privadas¶
Variáveis de instância “privadas”, que não podem ser acessadas, ​​exceto em métodos do próprio objeto, não existem em Python. No entanto, existe uma convenção que é seguida pela maioria dos programas em Python: um nome prefixado com um sublinhado (por exemplo: _spam ) deve ser tratado como uma parte não-pública da API (seja uma função, um método ou um atributo de dados). Tais nomes devem ser considerados um detalhe de implementação e sujeito a alteração sem aviso prévio.

Uma vez que existe um caso de uso válido para a definição de atributos privados em classes (especificamente para evitar conflitos com nomes definidos em subclasses), existe um suporte limitado a identificadores privados em classes, chamado desfiguração de nomes. Qualquer identificador no formato __spam (pelo menos dois sublinhados no início, e no máximo um sublinhado no final) é textualmente substituído por _classname__spam, onde classname é o nome da classe atual com sublinhado(s) iniciais omitidos. Essa desfiguração independe da posição sintática do identificador, desde que ele apareça dentro da definição de uma classe.

A desfiguração de nomes é útil para que subclasses possam sobrescrever métodos sem quebrar invocações de métodos dentro de outra classe. Por exemplo:

In [1]:
from datetime import datetime
import pytz


class ContaCorrente:    

    @staticmethod
    def _data_hora():
        fuso_BR = pytz.timezone('Brazil/East')
        horario_BR = datetime.now(fuso_BR)
        return horario_BR.strftime('%d/%m/%Y %H:%M:%S')

    def __init__(self, nome, cpf, agencia, num_conta):
        self._nome = nome
        self._cpf = cpf
        self._saldo = 0
        self._limite = None
        self._agencia = agencia
        self._num_conta = num_conta
        self._transacoes = []
        
    def consultar_saldo(self): #Método consultar saldo
        print(f"Seu saldo atual é de R$: {(self._saldo):,.2f}")
        
    def depositar(self, valor):
        self._saldo += valor
        self.consultar_saldo()
        self._transacoes.append((valor, f'Saldo R$ {(self._saldo):,.2f} às {ContaCorrente._data_hora()}'))
        
    def _limite_conta(self): # _nome_método significa método privado!
        self._limite = 500
        return self._limite
    
    def sacar_dinheiro(self, valor): 
        if self._saldo - valor < self._limite_conta():
            print("Você não tem saldo suficiente para sacar esse valor")
            self.consultar_saldo()
        else:
            self._saldo -= valor
            self._transacoes.append((- valor, f'Saldo {self._saldo} às {ContaCorrente._data_hora()}'))
            
    def consultar_limite_cheque_especial(self):
        print(f"Seu limite de cheque especial é de R$ {(self._limite_conta()):,.2f} Reais.")
        
    def consultar_historico_transacoes(self):
        print("Historico de transações:")
        for transacao in self._transacoes:
            print("Valor, Saldo, Data e Hora")
            print(transacao)
    
    def transferir(self, valor, conta_destino):
        self._saldo -= valor
        self._transacoes.append((- valor, self._saldo, ContaCorrente._data_hora()))
        conta_destino._saldo += valor
        conta_destino._transacoes.append((valor, conta_destino._saldo, ContaCorrente._data_hora()))

In [2]:
#programa

conta_leandro = ContaCorrente("Leandro", "111-222-333-45", 1234, 56789)

#print(conta_leandro.saldo)
conta_leandro._saldo = 2000
conta_leandro.consultar_saldo()

conta_leandro.depositar(10)

conta_leandro.consultar_historico_transacoes()

conta_amigo = ContaCorrente("Gustavo", '222-333-444-56', 5555, 777777)

print(conta_amigo._nome)
conta_amigo.consultar_saldo()

Seu saldo atual é de R$: 2,000.00


In [3]:
conta_leandro.depositar(10)

Seu saldo atual é de R$: 2,010.00


In [4]:
conta_leandro.consultar_historico_transacoes()

Historico de transações:
Valor, Saldo, Data e Hora
(10, 'Saldo R$ 2,010.00 às 24/12/2022 13:26:54')


In [5]:
conta_amigo = ContaCorrente("Gustavo", '222-333-444-56', 5555, 777777)

In [6]:
print(conta_amigo._nome)
conta_amigo.consultar_saldo()

Gustavo
Seu saldo atual é de R$: 0.00


In [7]:
#método transferir
conta_leandro.transferir(1010, conta_amigo)

In [8]:
conta_leandro.consultar_saldo()
print(conta_amigo._nome)
conta_amigo.consultar_saldo()
print("\033[34m-" *20)
conta_leandro.consultar_historico_transacoes()
print("\033[34m-" *20)
conta_amigo.consultar_historico_transacoes()

Seu saldo atual é de R$: 1,000.00
Gustavo
Seu saldo atual é de R$: 1,010.00
--------------------
Historico de transações:
Valor, Saldo, Data e Hora
(10, 'Saldo R$ 2,010.00 às 24/12/2022 13:26:54')
Valor, Saldo, Data e Hora
(-1010, 1000, '24/12/2022 13:26:54')
--------------------
Historico de transações:
Valor, Saldo, Data e Hora
(1010, 1010, '24/12/2022 13:26:54')


In [9]:
print(conta_leandro._agencia)

1234
